In [1]:
import numpy
from openfermion.utils import givens_decomposition

from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

In [2]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:3, :]
print("The matrix Q:")
print(Q)
print()

# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print("The sequence of parallel givens rotations:")
for parallel_set in givens_rotations:
    print(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi) for i, j, theta, phi in parallel_set])
print()

# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print("The matrix VQ:")
print(W)
print()

# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

print("The matrix VQU^\dagger:")
print(V.dot(Q.dot(U.T.conj())))
print()

# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("VQU^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

The matrix Q:
[[-0.218-0.159j  0.024+0.488j  0.261-0.398j -0.530+0.024j  0.133-0.352j  0.156+0.117j]
 [-0.474-0.244j -0.187+0.255j  0.244-0.052j  0.425+0.17j  -0.323+0.394j  0.222+0.186j]
 [ 0.367-0.12j  -0.198+0.176j -0.016+0.144j  0.007+0.073j -0.036-0.012j  0.681-0.537j]]

The sequence of parallel givens rotations:
['2, 3, 0.253, 0.633']
['1, 2, 0.760, 0.841', '3, 4, 0.770, -0.624']
['0, 1, 0.922, 1.020', '2, 3, 1.423, 1.207', '4, 5, 1.523, 1.239']
['1, 2, 1.159, 1.336', '3, 4, 1.340, 2.474']
['2, 3, 1.295, -1.504']

The matrix VQ:
[[-0.585-0.151j  0.170+0.552j  0.274-0.455j -0.127+0.053j -0.000-0.j    -0.000+0.j   ]
 [ 0.315+0.045j  0.061+0.068j -0.014-0.144j -0.681-0.086j  0.298-0.557j  0.000+0.j   ]
 [-0.001+0.199j -0.254+0.049j -0.004-0.083j -0.039+0.074j -0.064+0.018j  0.749+0.56j ]]

Number of rotations to perform in parallel: 1
[[-0.585-0.151j  0.170+0.552j  0.283-0.47j   0.000-0.j    -0.000-0.j    -0.000+0.j   ]
 [ 0.315+0.045j  0.061+0.068j  0.136-0.223j -0.584+0.287j  0.29